In [ ]:
#!pip install folium
#asenna tarvittaessa

Alempi koodi tekee kartan jossa kaivonkansien kuvat "thumpnailina" voi olla raskas pyörittää, jos paljon dataa

In [ ]:
import pandas as pd
import folium
import ast
from PIL import Image
import io
import base64
from PIL import Image, ImageOps

# Read your data
data = pd.read_csv("updated_predictions.csv")

# Function to convert DMS to Decimal Degrees
def dms_to_dd(dms):
    degrees, minutes, seconds = dms
    return degrees + (minutes / 60) + (seconds / 3600)

# Parse coordinate strings into tuples and convert to decimal degrees
data['Latitude'] = data['Latitude'].apply(lambda x: dms_to_dd(ast.literal_eval(x)) if pd.notna(x) else None)
data['Longitude'] = data['Longitude'].apply(lambda x: dms_to_dd(ast.literal_eval(x)) if pd.notna(x) else None)
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Create map centered around the average location
map_center = [data['Latitude'].mean(), data['Longitude'].mean()]
map = folium.Map(location=map_center, zoom_start=12)

# Function to convert image to base64 and resize with higher quality
def image_to_base64(image_path, size=(500, 500)):  # Adjusted size for better clarity
    try:
        with Image.open(image_path) as img:
            # Resize the image using high-quality resampling
            img = ImageOps.fit(img, size, Image.LANCZOS)  # Use LANCZOS for better quality resizing
            byte_arr = io.BytesIO()
            img.save(byte_arr, format='JPEG', quality=90)  # Higher quality setting
            return base64.b64encode(byte_arr.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"Failed to load or convert image {image_path}: {e}")
        return None

# Function to add markers with embedded images
def add_marker(row):
    #relative_path_to_images = "../kaivot/kaivot2" #without red rectangle around manholecover.
    relative_path_to_images = "runs/detect/exp" #Update the path to your images
    image_path = f"{relative_path_to_images}/{row['Image Name']}"
    image_base64 = image_to_base64(image_path)
    if image_base64:
        image_html = f'<img src="data:image/jpeg;base64,{image_base64}" alt="Image not found" width="100">'
        popup_html = folium.Popup(folium.Html(image_html + f"<br><b>{row['Image Name']}</b>: {row['Prediction']}", script=True), max_width=250)
        folium.Marker([row['Latitude'], row['Longitude']], popup=popup_html).add_to(map)
    else:
        print(f"Image not found for {row['Image Name']}")

# Apply the marker adding function to each row in batches
for start in range(0, len(data), 100):  # Adjust batch size based on memory and performance
    data.iloc[start:start+100].apply(add_marker, axis=1)

# Save the map to an HTML file
map.save('image_map.html')
print("Map has been created and saved as image_map.html")


Seuraava koodi tekee hieman kevyemmän kartan plottauksen ilman "thumpnaileja"

In [ ]:
import pandas as pd
import folium
import ast

# Read your data
data = pd.read_csv("updated_predictions.csv")

# Function to convert DMS to Decimal Degrees
def dms_to_dd(dms):
    degrees, minutes, seconds = dms
    return degrees + (minutes / 60) + (seconds / 3600)

# Parse coordinate strings into tuples and convert to decimal degrees
data['Latitude'] = data['Latitude'].apply(lambda x: dms_to_dd(ast.literal_eval(x)) if pd.notna(x) else None)
data['Longitude'] = data['Longitude'].apply(lambda x: dms_to_dd(ast.literal_eval(x)) if pd.notna(x) else None)
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Create map centered around the average location
map_center = [data['Latitude'].mean(), data['Longitude'].mean()]
map = folium.Map(location=map_center, zoom_start=12, control_scale=False)  # control_scale=False removes the scale at the corner

# Function to add simple markers with text labels
def add_marker(row):
    popup_text = f"{row['Image Name']}: {row['Prediction']}"
    folium.Marker([row['Latitude'], row['Longitude']],
                  popup=folium.Popup(popup_text, max_width=250)).add_to(map)

# Apply the marker adding function to each row
data.apply(add_marker, axis=1)

# Save the map to an HTML file
map.save('image_map2.html')
print("Map has been created and saved as image_map2.html")
